In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
#                   zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

In [9]:
from scipy.interpolate import RectBivariateSpline
from skimage.transform import resize

In [10]:
patches_rootdir = '/home/yuncong/CSHL_data_patches'

In [11]:
# scoremaps_rootdir = '/home/yuncong/CSHL_scoremaps_lossless_inceptionModel'
scoremaps_rootdir = '/home/yuncong/CSHL_scoremaps_lossless'
if not os.path.exists(scoremaps_rootdir):
    os.makedirs(scoremaps_rootdir)
    
# predictions_rootdir = '/home/yuncong/CSHL_patch_predictions_inceptionModel'
predictions_rootdir = '/home/yuncong/CSHL_patch_predictions'
if not os.path.exists(predictions_rootdir):
    os.makedirs(predictions_rootdir)

In [12]:
for stack in ['MD585']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD594]:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590']:

    first_bs_sec, last_bs_sec = section_range_lookup[stack]

    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
    indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

#     for sec in range(188, last_detect_sec+1):
    for sec in range(first_detect_sec, last_detect_sec+1):
#     for sec in [142]:

        if sec not in indices_allROIs_allSections.columns:
            continue

        print sec

        indices_roi = indices_allROIs_allSections[sec]['roi1']

        predictions_dir = os.path.join(predictions_rootdir, stack, '%04d'%sec)
        if not os.path.exists(predictions_dir):
            os.makedirs(predictions_dir)

        scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)
        if not os.path.exists(scoremaps_dir):
            os.makedirs(scoremaps_dir)

        ## define grid, generate patches

        t = time.time()

        dm = DataManager(stack=stack, section=sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]
        sample_locations_roi = sample_locations[indices_roi]
        
        ## interpolate

        interpolation_xmin, interpolation_ymin = sample_locations_roi.min(axis=0)
        interpolation_xmax, interpolation_ymax = sample_locations_roi.max(axis=0)
        interpolation_w = interpolation_xmax - interpolation_xmin + 1
        interpolation_h = interpolation_ymax - interpolation_ymin + 1

        ##### sample_locations_roi + scores to dense_score_map #####

        shrink_factor = 4 # do interpolation on a smaller grid, then resize to original dimension

        sample_locations_unique_xs = np.unique(sample_locations_roi[:,0])
        sample_locations_unique_ys = np.unique(sample_locations_roi[:,1])

        n_sample_x = sample_locations_unique_xs.size
        n_sample_y = sample_locations_unique_ys.size

        index_x = dict([(j,i) for i,j in enumerate(sample_locations_unique_xs)])
        index_y = dict([(j,i) for i,j in enumerate(sample_locations_unique_ys)])
        sample_location_indices = np.asarray([(index_x[x], index_y[y]) for x, y in sample_locations_roi])

        sample_locations_interpolatedArea_ys_matrix, \
        sample_locations_interpolatedArea_xs_matrix = np.meshgrid(range(interpolation_ymin/shrink_factor, 
                                                                        interpolation_ymax/shrink_factor), 
                                                                  range(interpolation_xmin/shrink_factor, 
                                                                        interpolation_xmax/shrink_factor), 
                                                                  indexing='ij')


        sys.stderr.write('preprocess: %.2f seconds\n' % (time.time() - t))
        
        dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}
        probs = np.load(predictions_dir + '/%(dataset)s_predictions.npy'% {'dataset': dataset})

        
        def generate_score_map(l):

#             label = labels[(l+1)%len(labels)]
            label = labels[l]

            if label == 'BackG':
                return

            score_matrix = np.zeros((n_sample_x, n_sample_y))
            score_matrix[sample_location_indices[:,0], sample_location_indices[:,1]] = probs[:,l]

            
            spline = RectBivariateSpline(sample_locations_unique_xs/shrink_factor, 
                                         sample_locations_unique_ys/shrink_factor, 
                                         score_matrix, 
                                         bbox=[interpolation_xmin/shrink_factor, 
                                               interpolation_xmax/shrink_factor, 
                                               interpolation_ymin/shrink_factor, 
                                               interpolation_ymax/shrink_factor])

            dense_score_map = spline.ev(sample_locations_interpolatedArea_xs_matrix, 
                                        sample_locations_interpolatedArea_ys_matrix)

            dense_score_map = resize(dense_score_map, (interpolation_h, interpolation_w))

            dense_score_map[dense_score_map < 1e-2] = 0
            
            if np.count_nonzero(dense_score_map[::8, ::8] > .1) < 1e3:
                sys.stderr.write('No %s is detected on section %d\n' % (label, sec))
                return
            
#             dense_score_map_lossless = np.pad(dense_score_map, ((interpolation_ymin, h-interpolation_ymax-1), 
#                                                   (interpolation_xmin, w-interpolation_xmax-1)),
#                                               mode='constant', constant_values=0)
            
#             t = time.time()
            bp.pack_ndarray_file(dense_score_map.astype(np.float32), 
                                   os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s.bp' % \
                                                {'dataset': dataset, 'label': label}))
    
#             filters = Filters(complevel=9, complib='blosc')
#             with open_file( os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s.hdf' % \
#                                                             {'dataset': dataset, 'label': label}), mode="w") as f:
#                 _ = f.create_carray('/', 'data', Atom.from_dtype(dense_score_map_lossless.dtype), filters=filters, obj=dense_score_map_lossless)
    
            np.savetxt(os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': label}),
                   np.array((interpolation_xmin, interpolation_xmax, interpolation_ymin, interpolation_ymax))[None], 
                   fmt='%d')
    
#             sys.stderr.write('write: %.2f seconds\n' % (time.time() - t))
        
#             return dense_score_map_lossless

        
        t = time.time()

        n_labels = probs.shape[1]

        _ = Parallel(n_jobs=12)(delayed(generate_score_map)(l) for l in range(n_labels))

        sys.stderr.write('interpolate: %.2f seconds\n' % (time.time() - t))

132


preprocess: 4.07 seconds
interpolate: 17.33 seconds
preprocess: 4.10 seconds
No Sol is detected on section 132
No RtTg is detected on section 132


133


interpolate: 49.18 seconds
preprocess: 4.13 seconds
No 12N is detected on section 133
No 6N is detected on section 133
No Tz is detected on section 133
No RtTg is detected on section 133


134


...

interpolate: 20.52 seconds
preprocess: 4.06 seconds
No 6N is detected on section 260
No 12N is detected on section 260
No RtTg is detected on section 260


261


interpolate: 42.17 seconds
preprocess: 4.16 seconds
No 12N is detected on section 261
No 7N is detected on section 261
No 5N is detected on section 261
No SuVe is detected on section 261
No Gr is detected on section 261
No LVe is detected on section 261
No 7n is detected on section 261
No Pn is detected on section 261
No VLL is detected on section 261
No 6N is detected on section 261
No Amb is detected on section 261
No R is detected on section 261
No Tz is detected on section 261
No Sol is detected on section 261
No RtTg is detected on section 261
No LRt is detected on section 261
No LC is detected on section 261
No AP is detected on section 261
No sp5 is detected on section 261


262


JoblibIOError: JoblibIOError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

...................................................................